# Masterboot image with Signature Provider for rt5xx

This notebook describes how to use a custom remote signing service for generating masterboot image using *nxpimage* tool.


In [1]:
%run ../../init_notebook.ipynb

import os
import pprint

pp = pprint.PrettyPrinter(indent=4)

plugins_dir = "../common/plugins/"

SASP_PLUGIN = os.path.join(plugins_dir, "sasp.py")
WORKSPACE = "workspace/mbimg/"  # change this to path to your workspace
DATA_DIR = "data_mbimg/"  # change this to path to your workspace
VERBOSITY = (
    "-v"  # verbosity of commands, might be -v or -vv for debug or blank for no additional info
)

env: JUPYTER_SPSDK=1
Created `%!` as an alias for `%execute`.


## Signature Provider Plugin

First, we need to setup the Signature Provider plugin and start the custom HSM. In order to do that, open signature provider notebook (signature_provider.ipynb) and follow the instructions there. Once you are done, come back and continue here.

## Config File Setup

The masterboot configuration file will be needed for successful generation of Masterboot image(MBI) using *nxpimage* application.
*
There are three types of MBI for RT5xx based on the authentication type: `Plain`, `CRC` and `Signed`.

In this example we will be interested only in the `Signed` image type.

Run the following code and see how the generated configuration template looks like.

In [1]:
import os

from spsdk.utils.misc import load_file

# choose family for the MCU
FAMILY = "mimxrt595s"

%! nxpimage $VERBOSITY mbi get-templates --family $FAMILY --output $WORKSPACE --force

CONFIG_PATH = os.path.join(WORKSPACE, "mimxrt595s_xip_signed.yaml")
# just for verification that the template was generated
assert os.path.exists(CONFIG_PATH)

config_content = load_file(CONFIG_PATH)
print(config_content)

nxpimage -v mbi get-templates --family mimxrt595s --output workspace/mbimg/ --force 
Creating workspace\mbimg\mimxrt595s_xip_plain.yaml template file.
Creating workspace\mbimg\mimxrt595s_xip_crc.yaml template file.
Creating workspace\mbimg\mimxrt595s_xip_signed.yaml template file.
Creating workspace\mbimg\mimxrt595s_load_to_ram_plain.yaml template file.
Creating workspace\mbimg\mimxrt595s_load_to_ram_crc.yaml template file.
Creating workspace\mbimg\mimxrt595s_load_to_ram_signed.yaml template file.
Creating workspace\mbimg\mimxrt595s_load_to_ram_encrypted.yaml template file.
# =============  Master Boot Image Configuration template for mimxrt595s:latest, Plain Signed XIP Image.  ==============

# ======================================================================================================================
#                                                 == General Options ==                                                 
# =====================================================

The generated configuration file contains all possible configuration settings. 

Some of them are required (labeled with `[Required]` comment), some of them are conditionally required (labeled with `[Conditionally required]` comment) and some are optional (labeled with `[Optional]` comment).

Let's now focus on two configuration settings: `signPrivateKey` and `signProvider`. These two configuration values are mutually exclusive, so only one can be chosen. 
In order to use Signature Provider, we will remove the line with `signPrivateKey`  and update the one with `signProvider`. 

Keep in mind that the signature provider configuration must meet following rules:
  - Configuration key
    - key names `sign_provider` or `signProvider` are allowed

  - Configuration value
    - format `"type=<identifier>;<key1>=<value1>;<key2>=<value2>;..."`
    - the `<identifier>` has to match the `identifier` class attribute defined in the custom signature provider(`plugins/sasp.py`)
    - the remaining key-value pairs are passed to the `__init__` method of the concrete Signature Provider
    - e.g.: `"type=file;file_path=private_key.pem"` will instantiate `spsdk.crypto.PlainFileSP(file_path='private_key.pem')`

Run the code bellow so you use only configuration settings related to this example.

In [1]:
import shutil

# Copy additional files needed for masterboot image creation
shutil.rmtree(WORKSPACE)
shutil.copytree(DATA_DIR, WORKSPACE)

pp.pprint(f"All files are ready in folder '{WORKSPACE}'")

"All files are ready in folder 'workspace/mbimg/'"


## Execution

At this point, we have everything we need to run nxpimage application using remote HSM for image signing.

In [1]:
%! nxpimage $VERBOSITY mbi export --plugin $SASP_PLUGIN --config $CONFIG_PATH

# check if the signed image exists
output_file = os.path.join(WORKSPACE, "my_mbi.bin")
assert os.path.exists(output_file)

nxpimage -v mbi export --plugin ../common/plugins/sasp.py --config workspace/mbimg/mimxrt595s_xip_signed.yaml 
RKTH: db31d46c717711a8231cbc38b1de8a6e8657e1f733e04c2ee4b62fcea59149fa
INFO:spsdk.apps.nxpimage:
+==0x0000_0000= Application Block ======+
|       Size: 14.5 kiB; 14,804 B        |
|+==0x0000_0000= Application ==========+|
||      Size: 13.0 kiB; 13,264 B       ||
|+==0x0000_33cf========================+|
|+==0x0000_33d0= Certification Block ==+|
||       Size: 1.3 kiB; 1,284 B        ||
|+==0x0000_38d3========================+|
|+==0x0000_38d4= RSA signature ========+|
||             Size: 256 B             ||
|+==0x0000_39d3========================+|
+==0x0000_39d3==========================+

Success. (Master Boot Image: workspace\mbimg\my_mbi.bin created.)


## HSM teardown

Last step is to stop custom HSM. In order to do that, open again the Signature provider notebook and stop the running jupyter notebook code cell.